#### Deriving the Stationarity Condition for an AR(1) Model

Let's see why stationarity imposes a condition on our model parameters. We'll use the AR(1) model for simplicity: $Y_t = c + \phi_1 Y_{t-1} + \epsilon_t$. For now, let's assume $c=0$ to make the math cleaner.

$$Y_t = \phi_1 Y_{t-1} + \epsilon_t$$

We can express $Y_{t-1}$ in terms of $Y_{t-2}$:

$$Y_{t-1} = \phi_1 Y_{t-2} + \epsilon_{t-1}$$

Now, let's substitute this back into the equation for $Y_t$:

$$Y_t = \phi_1 (\phi_1 Y_{t-2} + \epsilon_{t-1}) + \epsilon_t = \phi_1^2 Y_{t-2} + \phi_1 \epsilon_{t-1} + \epsilon_t$$

If we keep substituting backwards in time, we get:

$$Y_t = \phi_1^k Y_{t-k} + \sum_{i=0}^{k-1} \phi_1^i \epsilon_{t-i}$$

If this process started a very long time ago (as $k \to \infty$), the term $\phi_1^k Y_{t-k}$ represents the "memory" of the very distant past. For the influence of the distant past to fade away and for the series to have a finite, constant variance, this term must go to zero. This only happens if:

$$|\phi_1| < 1$$

If $|\phi_1| < 1$, the model is **stationary**. The impact of past shocks decays over time.
*   If $\phi_1 = 1$, the model is a **random walk**, which is non-stationary. Shocks have a permanent effect.
*   If $|\phi_1| > 1$, the model is **explosive** and non-stationary. The series will diverge to infinity.

This condition, $|\phi_1| < 1$, is the stationarity condition for an AR(1) model. For a general AR(p) model, a similar (but more complex) condition exists related to the roots of a characteristic polynomial.

---

## Identifying the Order 'p': Using Statistical Tools

We now know what an AR(p) model is, but a critical question remains: **How do we choose the order `p`?** Do we look back 1 step, 5 steps, 10 steps?

Simply guessing is not a good strategy. Instead, we use statistical tools that analyze the correlation structure of the time series. The two most important tools are the Autocorrelation Function (ACF) and the Partial Autocorrelation Function (PACF).

### The Autocorrelation Function (ACF)

The ACF measures the correlation between a time series and its lagged versions. The ACF at lag $k$ is the correlation between $Y_t$ and $Y_{t-k}$.

For an AR(p) process, the ACF will show a **gradual decay**. Why? Because $Y_t$ is directly influenced by $Y_{t-1}$, which is influenced by $Y_{t-2}$, and so on. This creates an indirect chain of correlations that fades out slowly. For example, in an AR(1) model, $\text{Corr}(Y_t, Y_{t-k}) = \phi_1^k$. Since $|\phi_1|<1$, this correlation decays exponentially to zero as $k$ increases.

While the ACF is useful, its slow decay makes it hard to pinpoint the exact order `p`. This is where the PACF comes in.

### The Partial Autocorrelation Function (PACF): The Key Identifier

The PACF gives us the **direct relationship** between $Y_t$ and $Y_{t-k}$, after removing the influence of the intermediate lags ($Y_{t-1}, Y_{t-2}, \dots, Y_{t-k+1}$).

**Analogy:** Imagine we want to measure the influence of your grandparents' genetics on you.
*   The **ACF** is like measuring the total similarity, which includes the genes passed through your parents.
*   The **PACF** is like measuring the *direct* genetic influence of your grandparents, after statistically removing the part that already came through your parents.

This ability to isolate the direct effect is exactly what we need. For an **AR(p) model**, the PACF has a very distinct signature:

> For a pure AR(p) process, the PACF will be statistically significant for the first `p` lags and then will **cut off sharply** to zero for all subsequent lags.

This is because, by definition, an AR(p) model says that $Y_t$ is directly influenced only by its first `p` lags. The influence of $Y_{t-p-1}$ on $Y_t$ is already fully captured through $Y_{t-1}, \dots, Y_{t-p}$.

Therefore, to identify the order `p`, we will generate a PACF plot and look for the lag at which it cuts off.

## Putting It All Together: A Practical Python Example

Let's make these concepts concrete with Python. We will use the `statsmodels` library, a powerhouse for statistical modeling in Python, and `numpy` for data generation.

First, let's ensure we have the necessary libraries installed.
```bash
pip install numpy pandas matplotlib statsmodels
```

### Step 1: Generating Synthetic AR(2) Data

The best way to learn is to work with data where we know the true underlying process. Let's create a synthetic AR(2) process. Our model will be:

$$Y_t = 0.6 Y_{t-1} + 0.3 Y_{t-2} + \epsilon_t$$

Here, the true order is $p=2$, with $\phi_1 = 0.6$ and $\phi_2 = 0.3$.

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.ar_model import AutoReg

# Set a seed for reproducibility
np.random.seed(42)

# Define the AR(2) process coefficients
# The first element is for the AR(0) term (always 1)
# The next elements are the phi coefficients with a negative sign
ar_coeffs = [1, -0.6, -0.3]
# The MA coefficients are [1] as it's a pure AR process
ma_coeffs = [1]

# Create the ArmaProcess
ar_process = ArmaProcess(ar_coeffs, ma_coeffs)

# Generate 500 data points from the process
n_samples = 500
y = ar_process.generate_sample(nsample=n_samples)

# Create a pandas Series for easier handling
ts = pd.Series(y)

print("Synthetic AR(2) data generated.")
```

### Step 2: Visualizing the Data

It's always a good practice to plot the time series first to get a feel for its behavior.

```python
# Plot the generated time series
plt.figure(figsize=(12, 6))
plt.plot(ts)
plt.title('Synthetic AR(2) Time Series')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True)
plt.show()
```
You should see a series that fluctuates around a mean of zero, which is characteristic of a stationary process.

### Step 3: Analyzing ACF and PACF Plots

Now, let's use the ACF and PACF plots to see if we can correctly identify the order $p=2$.

```python
# Create a figure with two subplots
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

# Plot the ACF
plot_acf(ts, ax=ax[0])
ax[0].set_title('Autocorrelation Function (ACF)')

# Plot the PACF
plot_pacf(ts, ax=ax[1])
ax[1].set_title('Partial Autocorrelation Function (PACF)')

plt.tight_layout()
plt.show()
```
When you run this code, you will see two plots:
1.  **ACF Plot**: You will observe that the correlations start high and **decay gradually**. This tells us that an AR model is likely appropriate. The pattern is a mix of exponential decay and sine-wave, which is common for AR(p) processes where $p>1$.
2.  **PACF Plot**: Here is the key! You will see two significant spikes at lag 1 and lag 2. After lag 2, the correlations **cut off sharply** and fall within the blue shaded area (the confidence interval). This is the classic signature of an AR(2) process.

The PACF plot clearly tells us that we should choose $p=2$.

### Step 4: Fitting the AR Model with `statsmodels`

Now that we have identified the order, we can fit an AR(2) model to our data using `statsmodels.tsa.ar_model.AutoReg`.

```python
# Fit an AR(2) model to the data
# We specify the data 'ts' and the number of lags 'lags=2'
model = AutoReg(ts, lags=2)
ar_results = model.fit()

# Print the model summary
print(ar_results.summary())
```

The summary output is rich with information, but let's focus on the coefficients table:

```
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0157      0.043     -0.364      0.716      -0.100       0.069
y.L1           0.5843      0.044     13.299      0.000       0.498       0.670
y.L2           0.2783      0.044      6.321      0.000       0.192       0.365
```

Let's interpret this:
*   The `coef` column shows the estimated values for our parameters.
*   The coefficient for `y.L1` (which is $\hat{\phi}_1$) is `0.5843`. This is very close to our true value of `0.6`.
*   The coefficient for `y.L2` (which is $\hat{\phi}_2$) is `0.2783`. This is very close to our true value of `0.3`.
*   The p-values (`P>|z|`) for both coefficients are `0.000`, meaning they are statistically highly significant.

Our statistical tools successfully identified the correct model structure, and the `statsmodels` library accurately estimated the underlying parameters.

---

## Conclusion

We have taken a deep dive into the Autoregressive model. We started with the simple intuition that the past can predict the future and formalized this into the AR(p) equation. We established that for this model to be meaningful, the time series must be **stationary**, a condition ensuring that the underlying process is stable over time.

Most importantly, we learned a practical, data-driven method for selecting the model's key parameter, the order `p`. By analyzing the **Partial Autocorrelation Function (PACF)** and looking for a sharp cutoff, we can confidently determine how many steps into the past our model needs to look.

Finally, we walked through a complete example in Python, from generating data to identifying the model order and fitting it with `statsmodels`. This workflow is fundamental to time series analysis and serves as a building block for more complex models like the Moving Average (MA) and ARMA models, which we may explore next.

https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/
https://www.kaggle.com/code/prashant111/complete-guide-on-time-series-analysis-in-python
https://en.wikipedia.org/wiki/Autoregressive_model